# Import Block

In [ ]:
%pip install torch torchvision torchaudio python-dotenv lightning geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv numpy mlflow


# Global Variables

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

import os
from dotenv import load_dotenv

import sys
src_path = "../../src/"
sys.path.append(src_path)
from utils.fed_env import FederatedEnvironment
from utils.dataset import Dataset
from utils.target_types import DTarget


In [ ]:
PATH_TO_DOT_ENV = "../../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

MLFLOW_HOST = "localhost"


In [ ]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

MLFLOW_HOST_PORT = os.environ.get("MLFLOW_HOST_PORT")

In [ ]:
N = 30
M = 1
K = 1

BATCH_SIZE = 4096
LEARNING_RATE = 0.001 
LAYER_MULTIPYER = 2
RANDOM_STATE = 1
TEST_SIZE = 0.2
EPOCHS = 500
NUM_NODES_PER_EPOCH = 3
NUM_TRAINING_PER_EPOCH = 2

MLFLOW_URI = f"http://{MLFLOW_HOST}:{MLFLOW_HOST_PORT}"

In [ ]:
DATABASE_URL = URL.create(
    DATABASE_TYPE,
    username=POSTGRES_USER,
    password=POSTGRES_PASSWORD,  # plain (unescaped) text
    host=DATABASE_HOST,
    port=POSTGRES_HOST_PORT,
    database=DATABASE_NAME,
)
# engine = create_engine(DATABASE_URL)
# engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

# Run Federated Learning

In [ ]:

server = FederatedEnvironment(
    d_type=Dataset.LIGHTNING,
    d_target=DTarget.BOOLEAN,
    db_url=DATABASE_URL,
    mlflow_uri=MLFLOW_URI,
    n=N,
    m=M,
    k=K, 
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
)
server.run_phase_1(
    epochs=EPOCHS
)
server.fed_avg(
    epochs=EPOCHS, 
    num_nodes_per_epoch=NUM_NODES_PER_EPOCH,
    num_training_per_epoch=NUM_TRAINING_PER_EPOCH,
)